In [1]:
import os
import pydicom
import nibabel as nib
import numpy as np
from tqdm import tqdm  # 진행 상태 표시
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter

In [7]:
def find_dicom_folders(base_path, target_folder_names):
    dicom_folders = []
    failed_folders = []
    #아래 폴더들까지 재귀적 탐색 진행
    def recursive_search(current_path):
        for root, dirs, files in os.walk(current_path):
            for target_folder_name in target_folder_names:
                # 타겟 폴더 탐색
                if target_folder_name in dirs:
                    target_path = os.path.join(root, target_folder_name)
                    #print(f"🔍 [타겟 폴더 발견]: {target_path}")

                    # 타겟 폴더 내부 탐색 (여러 시점 폴더 포함 - target folder 내에 여러 시점의 데이터가 다시 들어가있는 경우작 존재함)
                    for sub_root, sub_dirs, sub_files in os.walk(target_path):
                        for sub_dir in sub_dirs:
                            sub_dir_path = os.path.join(sub_root, sub_dir)
                            dcm_files = [f for f in os.listdir(sub_dir_path) if f.lower().endswith('.dcm')]

                            if dcm_files:  # DICOM 파일 존재
                                dicom_folders.append(sub_dir_path)
                                print(f"[V] [DICOM 발견]: {sub_dir_path}")
                            else:
                                failed_folders.append(sub_dir_path)
                                print(f"[X] [DICOM 없음]: {sub_dir_path}")

    # 최상위 폴더부터 시작
    recursive_search(base_path)

    return dicom_folders, failed_folders


def dicom_to_nifti(dicom_folder, output_path):
    # DICOM 파일 읽기
    dicom_files = [os.path.join(dicom_folder, f) for f in os.listdir(dicom_folder) if f.lower().endswith('.dcm')]

    if not dicom_files:
        print(f"[X] [오류] {dicom_folder}에 DICOM 파일이 없습니다.")
        return
    
    try:
        slices = []
        x_positions = []

        for dcm_path in dicom_files:
            dcm = pydicom.dcmread(dcm_path)
            if hasattr(dcm, "ImagePositionPatient"):
                # 슬라이스 데이터와 X좌표 저장
                slices.append(dcm.pixel_array)
                x_positions.append(dcm.ImagePositionPatient[0])  # X축 좌표 저장

        # X축 기준으로 슬라이스 정렬
        sorted_indices = np.argsort(x_positions)
        sorted_slices = [slices[i] for i in sorted_indices]

        # 3D 볼륨 생성
        volume = np.stack(sorted_slices, axis=-1)  # HxWxD 형태로 스택
        # print(f"volume(original) : {volume.shape}")
        z,x,y = volume.shape
        m=256
        pad_x = (m-x)//2
        pad_y = (m-y)//2
        
        pad_volume = np.pad(
            volume,
            pad_width=((0,0),(pad_x,m-x-pad_x), (pad_y,m-y-pad_y)),
            mode='minimum'
        )
        # print(volume.shape)
            # Step 2: z 방향 슬라이싱
        # 볼륨을 [m, m, m]로 슬라이싱 (중간 z 축 선택)
        center_z = z // 2  # z의 중간 인덱스
        start_z = max(0, center_z - m // 2)  # 슬라이싱 시작 z
        end_z = min(z, start_z + m)  # 슬라이싱 종료 z

        # # 확인: z 크기가 m보다 작은 경우 패딩 추가 필요
        # if cropped_volume.shape[0] < m:
        volume = pad_volume[start_z:end_z, :, :]  # 결과: [m, m, m]
        #     pad_z = m - cropped_volume.shape[0]
        #     cropped_volume = np.pad(
        #         cropped_volume,
        #         pad_width=((0, pad_z), (0, 0), (0, 0)),
        #         mode='constant',
        #         constant_values=0
        #     )
        # print(volume.shape)
        # NIfTI 파일 저장
        affine = np.eye(4)  # 기본 affine 행렬 (필요 시 수정 가능)
        nifti_img = nib.Nifti1Image(volume, affine)
        nib.save(nifti_img, output_path)
        print(f"[V] [NIfTI 저장 완료]: {output_path}")
    except Exception as e:
        print(f"[X] [오류 발생]: {dicom_folder} - {e}")




# 1. 3D Volume화를 시킥 경우 -> 하나의 폴더로 싹 담고, 이후에 Dataframe 따로 이용해서 MMSE Score에 대한 정보 담기
# 2. Multi-View 이용할 경우 : 얘네는 적으니까 폴더 구조 그대로 유지해서 진행한다.
def convert_adni_dicom_to_nifti(base_path, target_folder_names, output_base_path):
    # Step 1: 대상 DICOM 폴더 찾기
    print("탐색시작")
    dicom_folders, failed_folders = find_dicom_folders(base_path, target_folder_names)
    print(f"[V] 총 {len(dicom_folders)}개의 DICOM 폴더를 찾았습니다.")
    print(f"[X] 총 {len(failed_folders)}개의 폴더에서 DICOM 파일을 찾을 수 없습니다.")
    # 디버깅용 코드입니다.
    print(failed_folders)
    
    # Patient, MRI 정보를 저장할 리스트
    records = []

    for dicom_folder in tqdm(dicom_folders, desc="DICOM → NIfTI 변환 진행"):
        # relative_path와 관련 정보 추출
        relative_path = os.path.relpath(dicom_folder, base_path)
        rp_arr = relative_path.split(os.sep)  # os.sep으로 플랫폼에 맞게 경로 분리
        patient_number = rp_arr[0]
        mri_number = rp_arr[-1]
        visdate=rp_arr[1]
        
        print('patient number:', patient_number)
        print('MRI number :', mri_number)
        
        # 정보 저장
        records.append({"Patient Number": patient_number, "VISDATE":visdate,"MRI Number": mri_number})
        
        # 출력 폴더 설정
        output_folder = os.path.join(output_base_path)  # 폴더 구조 없이 저장
        os.makedirs(output_folder, exist_ok=True)

        # 출력 파일명 설정
        output_file = os.path.join(output_folder, f"{patient_number},{mri_number}.nii.gz")
        print('Output file:', output_file)
        
        # DICOM → NIfTI 변환
        dicom_to_nifti(dicom_folder, output_file)

    # 실패한 폴더 출력
    if failed_folders:
        print("\n[X] [조건을 만족하지 못한 폴더]:")
        for folder in failed_folders:
            print(f" - {folder}")
    
    # DataFrame 생성 및 저장
    if records:
        df = pd.DataFrame(records)
        csv_path = os.path.join(output_base_path, "patient_mri_mapping.csv")
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"\n[V] Patient-MRI 매핑 데이터가 {csv_path}에 저장되었습니다.")

In [9]:
 # 최상위 ADNI 폴더 경로 설정
base_path = "E:\\MRI 파일\\ADNI4"

# 찾고자 하는 타겟 폴더 이름 리스트
target_folder_names = [
"Accelerated_Sagittal_MPRAGE",
"Sagittal_3D_Accelerated_MPRAGE",
"Accelerated_Sag_IR-FSPGR",
"Accelerated_Sagittal_IR-FSPGR",
"Sag_Accel_IR-FSPGR",
"Sagittal_3D_Accelerated_0_angle_MPRAGE",
"Accelerated_Sagittal_MPRAGE_Phase_A-P",
"Accelerated_Sagittal_MPRAGE__MSV21",
"Accelerated_Sagittal_MPRAGE__MSV21_",
"Accelerated_Sagittal_MPRAGE__MSV21_RPT",
"Accelerated_Sagittal_IR-FSPGR__MSV21",
"Accelerated_Sagittal_IR-FSPGR__MSV21_",
"Accelerated_Sagittal_IR-FSPGR__MSV21__RPT",
"Accelerated_Sagittal_IR-FSPGR__MSV22",
"Accelerated_Sagittal_IR-FSPGR__MSV22_",
"Sagittal_3D_Accelerated_MPRAGE_MSV21_"
"Accelerated_Sag_IR-FSPGR__MSV21_",
"Sagittal_3D_Accelerated_MPRAGE__MSV21_"
]

# NIfTI 파일 저장 경로
output_base_path = "./niis/ADNI4"

# 변환 함수 호출
convert_adni_dicom_to_nifti(base_path, target_folder_names, output_base_path)

탐색시작
[V] [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_10042\2024-02-13\Accelerated_Sagittal_MPRAGE__MSV21_\I10365678
[V] [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_10073\2024-04-24\Accelerated_Sagittal_MPRAGE__MSV21_\I10454329
[V] [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_10082\2024-04-30\Accelerated_Sagittal_MPRAGE__MSV21_\I10454163
[V] [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_10158\2024-07-11\Accelerated_Sagittal_MPRAGE__MSV21_\I10883407
[V] [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_4441\2024-04-23\Accelerated_Sagittal_MPRAGE__MSV21_\I10454353
[V] [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_4900\2024-02-07\Accelerated_Sagittal_MPRAGE__MSV21_\I10363378
[X] [DICOM 없음]: E:\MRI 파일\ADNI4\003_S_6014\2024-04-10\Accelerated_Sagittal_MPRAGE__MSV21_\2024-04-10_13_09_22.0
[V] [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_6014\2024-04-10\Accelerated_Sagittal_MPRAGE__MSV21_\I10454361
[V] [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_6256\2024-06-11\Accelerated_Sagittal_MPRAGE__MSV21_\I10666741
[V] [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_6258\2024-05-23\Accelerated_Sagittal_MPRA

DICOM → NIfTI 변환 진행:   0%|          | 0/243 [00:00<?, ?it/s]

patient number: 003_S_10042
MRI number : I10365678
Output file: ./niis/ADNI4\003_S_10042,I10365678.nii.gz


DICOM → NIfTI 변환 진행:   0%|          | 1/243 [00:01<04:03,  1.01s/it]

[V] [NIfTI 저장 완료]: ./niis/ADNI4\003_S_10042,I10365678.nii.gz
patient number: 003_S_10073
MRI number : I10454329
Output file: ./niis/ADNI4\003_S_10073,I10454329.nii.gz


...

[V] [NIfTI 저장 완료]: ./niis/ADNI4\941_S_6384,I10568433.nii.gz
patient number: 941_S_7051
MRI number : I10873191
Output file: ./niis/ADNI4\941_S_7051,I10873191.nii.gz


DICOM → NIfTI 변환 진행: 100%|█████████▉| 242/243 [13:48<00:03,  3.74s/it]

[V] [NIfTI 저장 완료]: ./niis/ADNI4\941_S_7051,I10873191.nii.gz
patient number: 941_S_7074
MRI number : I10283169
Output file: ./niis/ADNI4\941_S_7074,I10283169.nii.gz


DICOM → NIfTI 변환 진행: 100%|██████████| 243/243 [13:51<00:00,  3.42s/it]

[V] [NIfTI 저장 완료]: ./niis/ADNI4\941_S_7074,I10283169.nii.gz

[X] [조건을 만족하지 못한 폴더]:
 - E:\MRI 파일\ADNI4\003_S_6014\2024-04-10\Accelerated_Sagittal_MPRAGE__MSV21_\2024-04-10_13_09_22.0
 - E:\MRI 파일\ADNI4\033_S_10175\2024-07-26\Accelerated_Sagittal_MPRAGE__MSV21_\2024-07-26_12_12_41.0
 - E:\MRI 파일\ADNI4\033_S_10175\2024-07-26\Accelerated_Sagittal_MPRAGE__MSV21_\2024-07-26_12_57_30.0
 - E:\MRI 파일\ADNI4\035_S_7121\2024-03-12\Accelerated_Sagittal_MPRAGE__MSV21_\2024-03-12_15_00_53.0
 - E:\MRI 파일\ADNI4\035_S_7121\2024-03-12\Accelerated_Sagittal_MPRAGE__MSV21_\2024-03-12_15_07_19.0
 - E:\MRI 파일\ADNI4\041_S_5100\2024-04-09\Accelerated_Sagittal_MPRAGE__MSV21_\2024-04-09_13_34_51.0
 - E:\MRI 파일\ADNI4\041_S_5100\2024-04-09\Accelerated_Sagittal_MPRAGE__MSV21_\2024-04-09_13_47_29.0
 - E:\MRI 파일\ADNI4\041_S_6226\2024-04-09\Accelerated_Sagittal_MPRAGE__MSV21_\2024-04-09_12_18_42.0
 - E:\MRI 파일\ADNI4\041_S_6226\2024-04-09\Accelerated_Sagittal_MPRAGE__MSV21_\2024-04-09_12_24_58.0
 - E:\MRI 파일\ADNI4\941_S

In [10]:
 # 최상위 ADNI 폴더 경로 설정
base_path = "E:\\MRI 파일\\ADNI3"
# NIfTI 파일 저장 경로
output_base_path = "./niis/ADNI3"
# 변환 함수 호출
convert_adni_dicom_to_nifti(base_path, target_folder_names, output_base_path)

 # 최상위 ADNI 폴더 경로 설정
base_path = "F:\\MRI 파일\\ADNI3"
# NIfTI 파일 저장 경로
output_base_path = "./niis/ADNI3"
# 변환 함수 호출
convert_adni_dicom_to_nifti(base_path, target_folder_names, output_base_path)

탐색시작
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\003_S_6915\2021-03-19\Accelerated_Sagittal_MPRAGE\I1423364
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\003_S_6924\2021-03-30\Accelerated_Sagittal_MPRAGE\I1427086
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\003_S_6954\2021-05-26\Accelerated_Sagittal_MPRAGE\I1448643
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\003_S_6959\2021-06-16\Accelerated_Sagittal_MPRAGE\I1456305
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\003_S_6996\2021-08-23\Accelerated_Sagittal_MPRAGE\I1483860
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\003_S_7010\Accelerated_Sagittal_MPRAGE\I1495816
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\011_S_7028\2022-03-11\Accelerated_Sagittal_MPRAGE\I1555916
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\011_S_7048\2022-03-18\Accelerated_Sagittal_MPRAGE\I1558064
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\011_S_7112\2022-11-04\Accelerated_Sagittal_MPRAGE\I1639219
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\013_S_6970\2021-07-13\Accelerated_Sagittal_MPRAGE\I1469350
[V] [DICOM 발견]: E:\MRI 파일\ADNI3\013_S_7097\2022-09-28\Accelerated_Sagittal_MPRAGE\I1629558
[V] [

DICOM → NIfTI 변환 진행:   0%|          | 0/181 [00:00<?, ?it/s]

patient number: 003_S_6915
MRI number : I1423364
Output file: ./niis/ADNI3\003_S_6915,I1423364.nii.gz


DICOM → NIfTI 변환 진행:   1%|          | 1/181 [00:02<06:52,  2.29s/it]

[V] [NIfTI 저장 완료]: ./niis/ADNI3\003_S_6915,I1423364.nii.gz
patient number: 003_S_6924
MRI number : I1427086
Output file: ./niis/ADNI3\003_S_6924,I1427086.nii.gz


...

[V] [NIfTI 저장 완료]: ./niis/ADNI3\941_S_6854,I1428390.nii.gz
patient number: 941_S_6854
MRI number : I1547737
Output file: ./niis/ADNI3\941_S_6854,I1547737.nii.gz


DICOM → NIfTI 변환 진행: 100%|█████████▉| 2141/2142 [1:59:59<00:03,  3.48s/it]

[V] [NIfTI 저장 완료]: ./niis/ADNI3\941_S_6854,I1547737.nii.gz
patient number: 941_S_7106
MRI number : I1619403
Output file: ./niis/ADNI3\941_S_7106,I1619403.nii.gz


DICOM → NIfTI 변환 진행: 100%|██████████| 2142/2142 [2:00:03<00:00,  3.36s/it]

[V] [NIfTI 저장 완료]: ./niis/ADNI3\941_S_7106,I1619403.nii.gz

[X] [조건을 만족하지 못한 폴더]:
 - F:\MRI 파일\ADNI3\024_S_2239\2021-10-20\Accelerated_Sagittal_MPRAGE\2021-10-20_09_27_54.0
 - F:\MRI 파일\ADNI3\024_S_2239\2021-10-20\Accelerated_Sagittal_MPRAGE\2021-10-20_10_10_15.0
 - F:\MRI 파일\ADNI3\037_S_4410\2017-03-01\Accelerated_Sagittal_MPRAGE\2017-03-01_10_05_07.0
 - F:\MRI 파일\ADNI3\137_S_4520\2020-11-12\Accelerated_Sagittal_MPRAGE\2020-11-12_08_02_52.0
 - F:\MRI 파일\ADNI3\168_S_6371\2023-07-27\Accelerated_Sagittal_MPRAGE\2023-07-27_13_22_12.0
 - F:\MRI 파일\ADNI3\168_S_6371\2023-07-27\Accelerated_Sagittal_MPRAGE\2023-07-27_13_38_32.0
 - F:\MRI 파일\ADNI3\941_S_6570\2018-08-31\Accelerated_Sagittal_MPRAGE\2018-08-31_10_05_11.0

[V] Patient-MRI 매핑 데이터가 ./niis/ADNI3\patient_mri_mapping.csv에 저장되었습니다.
